In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

%matplotlib inline

In [149]:
nltk.download('wordnet')
nltk.download("punkt") 
ps= PorterStemmer()
wnl = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [150]:
# import chardet

# with open('../dataset/RATIO.csv', mode='rb') as file:
#     raw_data = file.read()
#     answer = chardet.detect(raw_data)

# answer

In [151]:
dataset1  = pd.read_csv('../dataset/RATIO.csv', encoding='MacRoman')
dataset2  = pd.read_csv('../dataset/Equal.csv',encoding='MacRoman')
dataset3  = pd.read_csv('../dataset/sentiment.csv', encoding='MacRoman')

C:\Users\USER\AppData\Local\Temp\ipykernel_22208\2591924512.py:3: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset3  = pd.read_csv('../dataset/sentiment.csv', encoding='MacRoman')


In [152]:
data = pd.concat([dataset1, dataset2, dataset3], ignore_index=True)
data= data[['Review', 'Summary', 'Sentiment']]
data = data.apply(lambda x : x.str.lower() )

In [153]:
data[['Review', 'Summary']].fillna(" ", inplace=True,)
data.dropna(subset=['Sentiment'], inplace=True)
data.replace("nan", " ", inplace=True)


C:\Users\USER\AppData\Local\Temp\ipykernel_22208\3575991364.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['Review', 'Summary']].fillna(" ", inplace=True,)


In [154]:
data.describe()

,Review,Summary,Sentiment
count,312301,312301,312301
unique,1420,103197,3
top,,good,positive
freq,30805,17670,173035


In [155]:

data['agg_review'] = data['Summary'].fillna(" ") +" " + data['Review'].fillna(" ")

In [156]:
df = data[['agg_review', 'Sentiment']]
df

,agg_review,Sentiment
0,problems 1 it gets auto defrost randomly anyti...,positive
1,freeze segment is tooo large so cooling segmen...,positive
2,warranty sticker is not present when its run d...,positive
3,everything is goodworking fine but it is silly...,positive
4,refrigerator working fine but top and side vie...,positive
...,...,...
312296,its very awesome productby every video makers ...,positive
312297,awesome product and clear sound just go it mus...,positive
312298,good perfect product!,positive
312299,lot of echo when recording slightly disappointed,negative


In [158]:
df.describe()

,agg_review,Sentiment
count,312301,312301
unique,120945,3
top,good,positive
freq,2449,173035


In [ ]:
sample_data = data['Product_Description'][0]

In [ ]:
sample_data

'The Web Designer\x89Ûªs Guide to iOS (and Android) Apps, today @mention 10 a.m! {link} #sxsw'

##PREPROCESS DATA

In [ ]:
data = data.drop(['Text_ID','Product_Type' ], axis=1)
data

,Product_Description,Sentiment
0,The Web DesignerÛªs Guide to iOS (and Android...,2
1,RT @mention Line for iPad 2 is longer today th...,2
2,Crazy that Apple is opening a temporary store ...,2
3,The lesson from Google One Pass: In this digit...,2
4,RT @mention At the panel: &quot;Your mom has a...,2
...,...,...
6359,Thanks @mention for hosting this event #sxsw #...,2
6360,"SXSW iPhone app is awesome, but iPad app crash...",1
6361,RT @mention RT @mention Download 20+ free trac...,2
6362,OH at Texas Social Media Awards: 'You don't ne...,2


In [ ]:
def preprocess_data_lemmatize(text):
    cleaned_data = re.sub(r"[^A-Za-z\s]","", text )
    cleaned_data = cleaned_data.lower()
    cleaned_data = word_tokenize(cleaned_data)
    cleaned_data = [wnl.lemmatize(word) for word in cleaned_data if word not in stopwords.words('english')]
    cleaned_data =" ".join(cleaned_data)
    return cleaned_data

    

In [ ]:
def preprocess_data_stem(text):
    cleaned_data = re.sub(r"[^A-Za-z\s]","", text )
    cleaned_data = cleaned_data.lower()
    cleaned_data = word_tokenize(cleaned_data)
    cleaned_data = [ps.stem(word) for word in cleaned_data if word not in stopwords.words('english')]
    cleaned_data =" ".join(cleaned_data)
    return cleaned_data

    

In [ ]:
data['lemma_cleaned_text'] = data['Product_Description'].apply(preprocess_data_lemmatize)



In [ ]:
data['stem_cleaned_text'] = data['Product_Description'].apply(preprocess_data_stem)

In [ ]:
data

,Product_Description,Sentiment,lemma_cleaned_text,stem_cleaned_text
0,The Web DesignerÛªs Guide to iOS (and Android...,2,web designer guide io android apps today menti...,web design guid io android app today mention l...
1,RT @mention Line for iPad 2 is longer today th...,2,rt mention line ipad longer today yesterday sx...,rt mention line ipad longer today yesterday sx...
2,Crazy that Apple is opening a temporary store ...,2,crazy apple opening temporary store austin tom...,crazi appl open temporari store austin tomorro...
3,The lesson from Google One Pass: In this digit...,2,lesson google one pas digital environment user...,lesson googl one pass digit environ user want ...
4,RT @mention At the panel: &quot;Your mom has a...,2,rt mention panel quotyour mom ipad designing b...,rt mention panel quotyour mom ipad design boom...
...,...,...,...,...
6359,Thanks @mention for hosting this event #sxsw #...,2,thanks mention hosting event sxsw sxswi iwanta...,thank mention host event sxsw sxswi iwantacr link
6360,"SXSW iPhone app is awesome, but iPad app crash...",1,sxsw iphone app awesome ipad app crash every t...,sxsw iphon app awesom ipad app crash everi tim...
6361,RT @mention RT @mention Download 20+ free trac...,2,rt mention rt mention download free track ment...,rt mention rt mention download free track ment...
6362,OH at Texas Social Media Awards: 'You don't ne...,2,oh texas social medium award dont need ask mot...,oh texa social media award dont need ask mothe...


##ENCODING

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
labels = data["Sentiment"]
feature = data['lemma_cleaned_text']

In [ ]:
x_train, x_test, y_train, y_test  = train_test_split(feature,labels, test_size=0.3, random_state=142)

USING BAG OF WORDS

In [ ]:
x_train

5562    rt rumor apple opening temporary store downtow...
2281    developer learn mobile accessibility sxsw andr...
6212    google launch major new social network called ...
4082    barry diller say youre silly write content one...
5417    yeah wasnt got couldnt resist wearing blue jea...
                              ...                        
1616           mention awesome traded sxsw last year ipad
6317    spotted google map street view car wonder ill ...
5146    rt mention sxsw uxdes check designing ipad int...
4607               google handing free pi pi day sxsw win
277     shocked see google showcasing blogger sxsw tho...
Name: lemma_cleaned_text, Length: 4454, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1000, ngram_range=(1,2))
feature_vec = vectorizer.fit_transform(x_train)

In [ ]:
print(vectorizer.get_feature_names_out())
print(feature_vec.toarray())

['aapl' 'aaron' 'aarpbulletin' ... 'zone' 'zuckerberg' 'zynga']


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


None


In [ ]:
feature_vec.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4454, 6707))